# Pandas Aggregation & Cleaning — Case (V2)

Bu notebook bir sonraki adımdır:  
Notebook-1'de (Methods & Indexing) veriyi tanıdık. Burada:

- Eksik değer stratejileri (silme/doldurma)
- Tarih dönüşümü (`StartDate`)
- Basit feature engineering
- Daha güçlü aggregation (`groupby`, `agg`)
- Mini EDA çıktıları

> Hedef: “temiz ama aşırı mühendislik olmayan” bir case tamamlamak.


In [1]:
import pandas as pd

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)


In [2]:
df_raw = pd.read_csv("../data/Lending-company.csv")
df = df_raw.copy()
df.head()


,LoanID,StringID,Product,CustomerGender,Location,Region,TotalPrice,StartDate,Deposit,DailyRate,TotalDaysYr,AmtPaid36,AmtPaid60,AmtPaid360,LoanStatus
0,1,LoanID_1,Product B,Female,Location 3,Region 2,17600.0,04/07/2018,2200,45,365,3221,4166,14621,Active
1,2,LoanID_2,Product D,Female,Location 6,Region 6,NaN,02/01/2019,2200,45,365,3161,4096,16041,Active
2,3,LoanID_3,Product B,Male,Location 8,Region 3,16600.0,08/12/2016,1000,45,365,2260,3205,16340,NaN
3,4,LoanID_4,Product A,Male,Location 26,Region 2,17600.0,NaN,2200,45,365,3141,4166,16321,Active
4,5,LoanID_5,Product B,Female,Location 34,Region 3,21250.0,28/10/2017,2200,55,365,3570,4745,14720,Active


## 1) Missing Values: Tespit ve Strateji

Önce eksikleri say, sonra hangi kolon için hangi strateji mantıklı karar ver.


In [3]:
missing = df.isna().sum().sort_values(ascending=False)
missing


LoanStatus        37
TotalPrice        25
Region             1
StartDate          1
LoanID             0
StringID           0
Product            0
CustomerGender     0
Location           0
Deposit            0
DailyRate          0
TotalDaysYr        0
AmtPaid36          0
AmtPaid60          0
AmtPaid360         0
dtype: int64

## 2) Type Fix: StartDate'i datetime'a çevirme (örnek)

> Not: Format bilinmiyorsa `errors="coerce"` ile güvenli dönüşüm yapılır.


In [4]:
df["StartDate"] = pd.to_datetime(df["StartDate"], errors="coerce")
df["StartDate"].head()


0   2018-04-07
1   2019-02-01
2   2016-08-12
3          NaT
4          NaT
Name: StartDate, dtype: datetime64[ns]

## 3) Basit Cleaning Örnekleri

- Kategorik eksikler: 'Unknown' gibi bir label ile doldurma
- Sayısal eksikler: median ile doldurma (örnek)


In [5]:
# Örnek: LoanStatus eksiklerini etiketle
if "LoanStatus" in df.columns:
    df["LoanStatus"] = df["LoanStatus"].fillna("Unknown")

# Örnek: TotalPrice eksiklerini median ile doldurma
if "TotalPrice" in df.columns:
    df["TotalPrice"] = df["TotalPrice"].fillna(df["TotalPrice"].median())

df.isna().sum().sort_values(ascending=False).head(10)


StartDate         641
Region              1
LoanID              0
StringID            0
Product             0
CustomerGender      0
Location            0
TotalPrice          0
Deposit             0
DailyRate           0
dtype: int64

## 4) Aggregation: Region x LoanStatus

Daha analitik bir özet tablo çıkaralım.


In [6]:
pivot = (
    df.groupby(["Region", "LoanStatus"])
      .agg(
          avg_total_price=("TotalPrice", "mean"),
          loan_count=("LoanID", "count")
      )
      .sort_values(by="loan_count", ascending=False)
)

pivot.head(20)


avg_total_price  loan_count
Region   LoanStatus                                   
Region 6 Finished Payment     18902.777778         180
         Active               20264.728682         129
Region 3 Finished Payment     19997.979798          99
         Active               20745.663265          98
Region 2 Active               19712.222222          90
         Finished Payment     18971.067416          89
Region 1 Finished Payment     19342.279412          68
         Active               20220.689655          58
Region 4 Finished Payment     18558.823529          34
         Active               21695.370370          27
Region 7 Finished Payment     18712.000000          25
         Active               19390.909091          22
Region 5 Finished Payment     17582.142857          14
Region 6 Unknown              18479.166667          12
Region 5 Active               18935.000000          10
Region 2 Unknown              18421.428571           7
Region 3 Unknown              17707.142857           7
Region 5 Unknown              16958.333333           6
Region 6 Blocked              17560.000000           5
Region 8 Finished Payment     16830.000000           5

## 5) Çıktı

- En yüksek ortalama TotalPrice olan ilk 5 region
- En yüksek loan_count olan ilk 5 region


In [7]:
top_avg = df.groupby("Region")["TotalPrice"].mean().sort_values(ascending=False).head(5)
top_cnt = df.groupby("Region")["LoanID"].count().sort_values(ascending=False).head(5)

top_avg, top_cnt


(Region
 Region 3     20265.655340
 Region 14    19950.000000
 Region 4     19872.177419
 Region 10    19707.142857
 Region 1     19670.801527
 Name: TotalPrice, dtype: float64,
 Region
 Region 6    326
 Region 3    206
 Region 2    188
 Region 1    131
 Region 4     62
 Name: LoanID, dtype: int64)

In [8]:
print("----- DATA AUDIT -----\n")

print("Shape:", df.shape)
print("\nMissing values:")
print(df.isna().sum())

print("\nDtypes:")
print(df.dtypes)

print("\nStartDate type:", df["StartDate"].dtype)

print("\nSample rows:")
df.head()

----- DATA AUDIT -----

Shape: (1043, 15)

Missing values:
LoanID              0
StringID            0
Product             0
CustomerGender      0
Location            0
Region              1
TotalPrice          0
StartDate         641
Deposit             0
DailyRate           0
TotalDaysYr         0
AmtPaid36           0
AmtPaid60           0
AmtPaid360          0
LoanStatus          0
dtype: int64

Dtypes:
LoanID                     int64
StringID                  object
Product                   object
CustomerGender            object
Location                  object
Region                    object
TotalPrice               float64
StartDate         datetime64[ns]
Deposit                    int64
DailyRate                  int64
TotalDaysYr                int64
AmtPaid36                  int64
AmtPaid60                  int64
AmtPaid360                 int64
LoanStatus                object
dtype: object

StartDate type: datetime64[ns]

Sample rows:


,LoanID,StringID,Product,CustomerGender,Location,Region,TotalPrice,StartDate,Deposit,DailyRate,TotalDaysYr,AmtPaid36,AmtPaid60,AmtPaid360,LoanStatus
0,1,LoanID_1,Product B,Female,Location 3,Region 2,17600.0,2018-04-07,2200,45,365,3221,4166,14621,Active
1,2,LoanID_2,Product D,Female,Location 6,Region 6,17600.0,2019-02-01,2200,45,365,3161,4096,16041,Active
2,3,LoanID_3,Product B,Male,Location 8,Region 3,16600.0,2016-08-12,1000,45,365,2260,3205,16340,Unknown
3,4,LoanID_4,Product A,Male,Location 26,Region 2,17600.0,NaT,2200,45,365,3141,4166,16321,Active
4,5,LoanID_5,Product B,Female,Location 34,Region 3,21250.0,NaT,2200,55,365,3570,4745,14720,Active


### Data Quality Note

StartDate kolonunda %61 oranında eksik veri bulunmaktadır.
Bu nedenle zaman bazlı analiz bu notebook kapsamında yapılmamıştır.

## Data Cleaning & Preparation Summary

Bu notebook’ta ham kredi verisi analize hazırlanmıştır.

### 1. Data Type Düzeltmeleri
- `StartDate` kolonu datetime formatına dönüştürülmüştür.
- Sayısal kolonlar uygun numeric tipte tutulmuştur.
- Kategorik kolonlar object tipinde bırakılmıştır.

### 2. Missing Value Analizi
- Veri seti 1043 gözlem ve 15 değişkenden oluşmaktadır.
- `StartDate` kolonu yaklaşık %61 oranında eksik değer içermektedir.
- `Region` kolonu minimal eksik değere sahiptir.
- Kritik sayısal kolonlarda (`TotalPrice`, `Deposit`, `DailyRate`) eksik gözlem bulunmamaktadır.

### 3. Missing Handling Kararı
- `StartDate` yüksek oranda eksik olduğu için zaman analizi kapsam dışı bırakılmıştır.
- Sayısal kolonlarda imputasyon gerekmemiştir.
- Veri gözlemleri agresif şekilde silinmemiştir.

### 4. Aggregation İşlemleri
- Region bazlı ortalama `TotalPrice` hesaplanmıştır.
- LoanStatus bazlı karşılaştırmalar yapılmıştır.
- groupby ve sort_values metodları kullanılmıştır.

### 5. Veri Durumu

Bu aşamada veri:

✔ Tip olarak tutarlı  
✔ Kritik kolonlarda eksiksiz  
✔ Görsel analiz için hazır  
✔ Modelleme öncesi temel EDA’ya uygun  

Ancak:

- Zaman serisi analizi için uygun değildir.
- Gelişmiş istatistiksel analiz yapılmamıştır.